In [252]:
#import libraries
from scipy import stats
import uproot
import matplotlib as mpl
import matplotlib.pyplot as plt 
import pandas as pd
import numpy as np
from mpl_toolkits import mplot3d
from sklearn.cluster import AgglomerativeClustering
from sklearn.linear_model import LinearRegression


In [ ]:

#Creating the noise file 
def open_noise(filename) :
    noise0=uproot.open(filename) #using uproot to open the root file 
    noise1=noise0['pixTree']
    noise=pd.DataFrame(noise1['fData'].array(library="np"))
    noise=noise[noise.duplicated(subset=['row','col'])==True]
    noise.drop_duplicates(subset=['row','col'],inplace=True)  #one consider a pixel tobe noisy xhen on a short run (<1minute), with no source it declares a hit more than one time.
    X_noise=np.array(noise.col+((noise.chipId-4)*1024)) #changing the coordinates 
    Y_noise=np.array(noise.row)
    return X_noise,Y_noise

#put in the Noise data 
Noise1=open_noise('../Noise1')
Noise2=open_noise('../Noise2')
Noise3=open_noise('../Noise3')
Noise4=open_noise('../Noise4')


In [ ]:
#open the root file and getting the tree (the root file are multinoiseScan_Date_Hour_RUNXX-B0-ladderX.root)
def open_file (filename):
    file=uproot.open(filename)

    file1=file['pixTree']
    Data=file1['fData'].array(library="np")
    data=pd.DataFrame(Data)
    data.col=((data.chipId-4)*1024)+data.col
    return data

data1=open_file("../Data1")
data2=open_file("../Data2")
data3=open_file("../Data3")
data4=open_file("../Data4")
fig, axes = plt.subplots(4)

display(data1)  #show the table with the different entries
axes[0].hist2d(data1.col,data1.row,bins=100,density=True) #plotting the data as a 2D histogram (at this point data from ladder 2 and 4 are flipped)
axes[1].hist2d(data2.col,data2.row,bins=100,density=True)
axes[2].hist2d(data3.col,data3.row,bins=100,density=True)
axes[3].hist2d(data4.col,data4.row,bins=100,density=True)


In [285]:

#remove noise 
def remove_noise (noise,data) :
        n_0=data[data.col.isin(noise[0]) & data.row.isin(noise[1])].index
        if len(n_0)>0:
            data.drop(index=n_0,inplace=True)
        return data 


data1_filtered=remove_noise(Noise1,data1)
data2_filtered=remove_noise(Noise2,data2)
data3_filtered=remove_noise(Noise3,data3)
data4_filtered=remove_noise(Noise4,data4)


In [ ]:

#Displaying the filtered data (Monitoring the experiment)
fig, axes = plt.subplots(4)
axes[0].hist2d(data1_filtered.col,data1_filtered.row,bins=[600,300],norm='symlog',cmin=1)
axes[1].hist2d(data2_filtered.col,data2_filtered.row,bins=[600,300],norm='symlog',cmin=1)
axes[2].hist2d(data3_filtered.col,data3_filtered.row,bins=[600,300],norm='symlog',cmin=1)
axes[3].hist2d(data4_filtered.col,data4_filtered.row,bins=[600,300],norm='symlog',cmin=1)
plt.show()

In [ ]:
#flipping the data from the even plane in y 
def flip_data_row (data) :
    data.row=524-data.row
    return data

flip_data_row(data2_filtered)
flip_data_row(data4_filtered)



In [ ]:
# First alignement of the telescope 
def track (Pos_1,Pos_2,z) :
    x=((z-Pos_1[2])/(Pos_2[2]-Pos_1[2]))+Pos_1[0]
    y=((z-Pos_1[2])/(Pos_2[2]-Pos_1[2]))+Pos_1[1]
    return [x,y]

#for this first alignement, we just consider the mean in x and y of the hits and we built track 2 by 2, then moving the other plane to be on this track
def alignement_telescope (DATA,Z) : 
    for i in range (len(DATA)) :
        for j in range (len(DATA)-1,i,-1) :
            for k in range (len(DATA)) :
                if k!=j and k!=i :
                    print(i,j,k)
                    print(Z[j]-Z[i])
                    Moved=track([np.mean(DATA[i].col),np.mean(DATA[i].row),Z[i]],[np.mean(DATA[j].col),np.mean(DATA[j].row),Z[j]],Z[k])
                    print(Moved[0],Moved[1])
                    print(np.mean(DATA[k].col)-Moved[0],np.mean(DATA[k].row)-Moved[1])
                    DATA[k].row=DATA[k].row+(Moved[1]-np.mean(DATA[k].row))
                    DATA[k].col=DATA[k].col+(Moved[0]-np.mean(DATA[k].col))


    return DATA


Z=[0,10.6,145,155.6] #coordinates of the differents planes of BANCO
hitchipId=8
DATA =[data1_filtered[data1_filtered.chipId==hitchipId],data2_filtered[data2_filtered.chipId==hitchipId],data3_filtered[data3_filtered.chipId==hitchipId],data4_filtered[data4_filtered.chipId==hitchipId]] #restriction to the useful chip
DATA1=alignement_telescope(DATA,Z) 


In [ ]:
#Displaying the data aligned

fig, axes = plt.subplots(4)
axes[0].hist2d(DATA1[0].col,DATA1[0].row,bins=[600,300],norm='symlog',cmin=1)
axes[1].hist2d(DATA1[1].col,DATA1[1].row,bins=[600,300],norm='symlog',cmin=1)
axes[2].hist2d(DATA1[2].col,DATA1[2].row,bins=[600,300],norm='symlog',cmin=1)
axes[3].hist2d(DATA1[3].col,DATA1[3].row,bins=[600,300],norm='symlog',cmin=1)

axes[0].axis('scaled')
axes[1].axis('scaled')
axes[2].axis('scaled')
axes[3].axis('scaled')


plt.show()

In [ ]:
#Plotting superposed
fig, axes = plt.subplots()
plt.hist2d(DATA1[0].col,DATA1[0].row,bins=[600,300],norm='symlog',cmap='Greens')
plt.hist2d(DATA1[1].col,DATA1[1].row,bins=[600,300],norm='symlog',cmap='Blues',alpha=0.2)
plt.hist2d(DATA1[2].col,DATA1[2].row,bins=[600,300],norm='symlog',cmap='Reds',alpha=0.2)
plt.hist2d(DATA1[3].col,DATA1[3].row,bins=[600,300],norm='symlog',cmap='viridis',alpha=0.2,cmin=1)
plt.axis('scaled')
plt.show()

In [294]:
#Clustering with Agglomerative Clustering (this method use a clustering method based on dichotomy )

def Clustering (data) :
    n=0
    Cluster=[]
    L=[]
    for i in range(max(data.trgNum)+1) :
        if data[data.trgNum==i].empty ==False :
            if (i in L) ==False :
                L.append(i)
                data_trgNum= data[data.trgNum==i] 
                if len(data_trgNum)!=1 :
                    X=data_trgNum[['row','col']].to_numpy()
                    clustering = AgglomerativeClustering(n_clusters=None,distance_threshold=2,metric='euclidean').fit(X)
                    labels = clustering.labels_ 
                    for i in labels :
                        Cluster.append(i+n)
                    n=max(labels)+1+n
                else :
                    Cluster.append(n)
                    n+=1
    return Cluster

Cluster1=Clustering(DATA1[0])
Cluster2=Clustering(DATA1[1])
Cluster3=Clustering(DATA1[2])
Cluster4=Clustering(DATA1[3])


In [295]:
#Setting the clusters to the right data

data1_clusterized = DATA1[0]
data1_clusterized['cluster']=Cluster1
data2_clusterized = DATA1[1]
data2_clusterized['cluster']=Cluster2
data3_clusterized = DATA1[2]
data3_clusterized['cluster']=Cluster3
data4_clusterized = DATA1[3]
data4_clusterized['cluster']=Cluster4


In [296]:
#computing barycenter

def reduce_by_barycenter (data) :
    data['Xbar'] = 0
    data['Ybar'] = 0
    for i in range (1,data.cluster.max()+1) :
        data.loc[data['cluster']==i,'Xbar']=data[data['cluster']==i].col.mean()
        data.loc[data['cluster']==i,'Ybar']=data[data['cluster']==i].row.mean()
    B=data.drop_duplicates(subset=['Xbar','Ybar','cluster'],keep='first',inplace=False)
    return B

#data redyuced to only the barycenter of the cluster 
data1_reduced=reduce_by_barycenter (data1_clusterized)
data2_reduced=reduce_by_barycenter (data2_clusterized)
data3_reduced=reduce_by_barycenter (data3_clusterized)
data4_reduced=reduce_by_barycenter (data4_clusterized)



In [297]:
#fit the data 
def chi2 (xexp,xfit): #computing chi2
    chi2 =0 
    for i in range (len(xexp)):
        chi2+=((xexp[i]-xfit[i])**2)/xexp[i]
    return chi2

def tracking (X,Y,Z0) : #getting the tracks we want and there score (we consider 2 going from 0 to 160, taking into account all the planes )
    model_X = LinearRegression()
    model_Y= LinearRegression()
    Z0=np.array(Z0)
    model_X.fit(Z0.reshape(-1,1),X)
    model_Y.fit(Z0.reshape(-1,1),Y)
    X_fit=model_X.predict(np.arange(0,160).reshape((-1, 1)))
    Y_fit=model_Y.predict((np.arange(0,160).reshape((-1, 1))))
    r_sq_X = model_X.score(Z0.reshape(-1,1),X)
    r_sq_Y=model_Y.score(Z0.reshape(-1,1),Y)
    chi2_x=chi2(X,[X_fit[0],X_fit[11],X_fit[146],X_fit[156]])
    chi2_y=chi2(Y,[Y_fit[0],Y_fit[11],Y_fit[146],Y_fit[156]])
    return ([X_fit,Y_fit],[r_sq_X,r_sq_Y],[chi2_x,chi2_y])

#computing residuals
def distance_from_fit (xexp,xfit):
    d=0 
    for i in range (len(xexp)):
        d+=(xexp[i]-xfit[i])/4
    return d


In [ ]:

#Alignement from the tracking : In this more precise alignement technique we compute all the residuals from a track (3 points fitting, distance from the track of the 4th one)

def precise_alignement (DATA,Z0) :
    Distances_X=[[],[],[],[]]
    Distances_Y=[[],[],[],[]]
    for i in range(max(data1.trgNum)) :
        if (len(DATA[0][DATA[0].trgNum==i]) ==1) and (len(DATA[1][DATA[1].trgNum==i])==1) and (len(DATA[2][DATA[2].trgNum==i]) ==1) and (len(DATA[3][DATA[3].trgNum==i]) ==1):
            for k in range(4):
                X=[]
                Y=[]
                Z=[]
                for j in range (4):
                    if j!=k:
                        X.append(DATA[j].at[DATA[j][DATA[j].trgNum==i],'Xbar'])
                        Y.append(DATA[j].at[DATA[j][DATA[j].trgNum==i],'Ybar'])
                        Z.append(int(Z0[j]))
                L=tracking (X,Y,Z)
                distance_x=L[0][0][int(Z0[k])]-DATA[k].at[DATA[k][DATA[k].trgNum==i],'Xbar']
                distance_y=L[0][1][int(Z0[k])]-DATA[k].at[DATA[k][DATA[k].trgNum==i],'Ybar']
                Distances_X[k].append(distance_x)
                Distances_Y[k].append(distance_y)
    return Distances_X,Distances_Y



DATA=[data1_reduced,data2_reduced,data3_reduced,data4_reduced]
Z=[0,10.6,145,155.6]
Distances=precise_alignement(DATA,Z)

In [ ]:
#moving from the correction that we had previously
def move (data,x,y):
    data.row=data.row+x
    data.col=data.col+y

for i in range(4) :
    move(DATA[i],np.mean(Distances[0][i]),np.mean(Distances[1][i]))



In [233]:
#computing the mean resolution (Distances from the track in X and in Y), useful to get the quality of the alignement

def mean_resolution (data1,data2,data3,data4,Z) :
    Resolutions_X =[]
    Resolutions_Y =[]

    Tracking=[]
    for i in range(max(data1.trgNum)) :
        if (len(data1[data1.trgNum==i]) ==1) and (len(data4[data4.trgNum==i])==1) and (len(data2[data2.trgNum==i]) ==1) and (len(data3[data3.trgNum==i]) ==1):
            X=[data1.at[data1[data1.trgNum==i],'Xbar'],data2.at[data2[data2.trgNum==i],'Xbar'],data3.at[data3[data3.trgNum==i],'Xbar'],data4.at[data4[data4.trgNum==i],'Xbar']]
            Y=[data1.at[data1[data1.trgNum==i],'Ybar'],data2.at[data2[data2.trgNum==i],'Ybar'],data3.at[data3[data3.trgNum==i],'Ybar'],data4.at[data4[data4.trgNum==i],'Ybar']]
            L=tracking (X,Y,Z)
            Resolutions_X.append(distance_from_fit(X,[L[0][0][0],L[0][0][11],L[0][0][146],L[0][0][156]]))
            Resolutions_Y.append(distance_from_fit(Y,[L[0][1][0],L[0][1][11],L[0][1][146],L[0][1][156]]))
            Tracking.append([L,i])
            if (distance_from_fit(X,[L[0][0][0],L[0][0][11],L[0][0][146],L[0][0][156]]))>150 :
                print(int(i))

    return(Resolutions_X,Resolutions_Y)
Resolutions=mean_resolution(DATA[0],DATA[1],DATA[2],DATA[3],Z)